In [159]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
tree = ET.parse('ADSP-BF706-registers.xml')
root = tree.getroot()

In [193]:
class reg(object):
    def __init__(self, attrib, children):
        self.attrib = attrib
        self.children = children
        
def printBits(row):
    fo.write("\t\tuint32_t " + row['name'] + ":" + str(row['bit-size']) + ";" + \
             "\t\t\t/*!< bit\t" + str(row['bit-position']) + \
             ( (".." + str(row['bit-position'] + (row['bit-size'] - 1))) if row['bit-size'] > 1 else "") + \
             "  " + row['description'] + "  */\n")
    
def printStructs(row):
    fo.write("\t" + ( "__IO" if row['type'] != 'None' else "\t") + \
             " " + row['reg-type'] + "\t\t" + row['reg-name'] + ";" + \
             "\t\t" + ( ("/**< \\brief " + row['description'] + " */") if row['type'] != 'None' else "" ) + "\n"\
            )
    
def addReserved(df, length):
    #add in reserved bits if necessary
    for ix, row in df.iterrows():
        if ix > 0:
            prev = df.iloc[ix - 1]
            if (prev['bit-position'] + prev['bit-size']) < row['bit-position']:
                #add in reserved
                df = df.append([{'bit-position' : (prev['bit-position'] + prev['bit-size']), \
                          'bit-size' : row['bit-position'] - (prev['bit-position'] + prev['bit-size']),\
                          'description' : 'Reserved',\
                          'name' : ''}])
                return df
            
    #check that we go all the way to the end
    last = df.iloc[-1]
    if last['bit-position'] + last['bit-size'] < length:
        #add in reserved
        df = df.append([{'bit-position' : (last['bit-position'] + last['bit-size']), \
                  'bit-size' : length - (last['bit-position'] + last['bit-size']),\
                  'description' : 'Reserved',\
                  'name' : ''}])
        return df
            
    return None

def addReservedRegs(df, reserved_index):
    #add in reserved bytes if necessary
    for ix, row in df.iterrows():
        if ix > 0:
            prev = df.iloc[ix - 1]
            if (prev['read-address'] + prev['bit-size']/8) < row['read-address']:
                #add in reserved
                df = df.append([{'type' : 'None', 'reg-type' : 'RoReg8', \
                              'reg-name' : "Reserved" + str(reserved_index) + "[" + format(row['read-address'] - (prev['read-address'] + prev['bit-size']/8), '#04x') + "]", \
                                'read-address' : prev['read-address'] + prev['bit-size']/8, \
                                'bit-size' : (row['read-address'] - (prev['read-address'] + prev['bit-size']/8)) * 8 }])
                return df
    return None

def cleanGroupName(g):
    name = ""
    try:
        name = g.split("(")[1].split(")")[0]
    except IndexError:
        name = g
    return name.replace(" ", "_")

In [194]:
regdefs = root[2]

groups = []

for r in regdefs.findall('register'):
    g = r.get('group')
    if not (g is None) and (not g in groups):
        groups.append(g)

# Open a file
fo = open("bf706_device.h", "wb")
fo.write( "#ifndef _BF706_DEVICE_\n#define _BF706_DEVICE_\n");

fo.write("""
#include <stdint.h>

#ifndef __IO
  #define       __IO    volatile
#endif

typedef volatile const uint8_t \tRoReg8;
""")

for g in groups:

    groupdefs = regdefs.findall(".register[@group='"+ g +"']")
    
    groupRegs = []

    for r in groupdefs:
        parent_name = r.get('name')
        try:
            reg_name = parent_name.split(g + "_")[1]
        except:
            reg_name = parent_name

        children = regdefs.findall(".register[@parent='"+ parent_name +"']")
        if children:
            df = DataFrame([c.attrib for c in children])
            #convert columns to numbers
            df[['bit-position','bit-size']] = df[['bit-position','bit-size']].apply(pd.to_numeric)

            #extract names
            df['name'] = df['name'].apply(lambda x: x.split(parent_name + ".")[1])

            df = df.drop('parent', axis=1)

            while True:
                #sort by bit position
                df = df.sort_values(['bit-position'])
                df = df.reset_index(drop=True)

                with_reserved = addReserved(df, int(r.attrib['bit-size']))
                if with_reserved is None:
                    break
                else:
                    df = with_reserved

        r.attrib['reg-name'] = reg_name
        r.attrib['reg-type'] = cleanGroupName(g) + '_' + reg_name + "_Type"

        regObj = reg(r.attrib, df)

        groupRegs.append(regObj)

    structGroup = DataFrame([c.attrib for c in groupRegs])
    structGroup['bit-size'] = structGroup['bit-size'].apply(lambda x: int(x))
    structGroup['read-address'] = structGroup['read-address'].apply(lambda x: int(x, 0))

    reserved_inserted = 1
    while True:
        #sort by address
        structGroup = structGroup.sort_values(['read-address'])
        structGroup = structGroup.reset_index(drop=True)

        with_reserved = addReservedRegs(structGroup, reserved_inserted)
        if with_reserved is None:
            break
        else:
            structGroup = with_reserved
            reserved_inserted = reserved_inserted + 1
            
    #write to the file

    for r in groupRegs:
        fo.write("/* ----- " + cleanGroupName(g) + "_" + r.attrib['reg-name'] + " : " + "(" + r.attrib['bit-size'] + ") " + \
                 r.attrib['description'] + " ----- */\n")
        fo.write("typedef union {\n")
        fo.write("\tstruct {\n")
        r.children.apply(printBits, axis=1)
        fo.write("\t} bit;\t\t/*!< Structure\tused for bit access\t\t*/\n")
        fo.write("\tuint32_t reg;\t\t/*!< Type\tused for bit access\t\t*/\n")
        fo.write("} " + r.attrib['reg-type'] + ";\n\n")


    #create full struct
    fo.write("typedef struct {\n")
    structGroup.apply(printStructs, axis=1)
    fo.write("} " + cleanGroupName(g).title() + ";\n")
    
#write pointers to structs
fo.write("\n\n /*------- INSTANCES -------*/\n\n")
for g in groups:
    lowest = None
    regs = regdefs.findall(".register[@group='"+ g +"']")
    for r in regs:
        if lowest == None or int(r.attrib['read-address'], 0) < int(lowest.attrib['read-address'], 0):
            lowest = r
            
    fo.write("#define " + cleanGroupName(g) + "\t\t\t((" + cleanGroupName(g).title() + "\t\t*)" + lowest.attrib['read-address'] + "UL)\n")

fo.write( "\n\n#endif");
fo.close()